In [ ]:
import xarray as xr
import rioxarray as rio
import glob
import os
import numpy as np
import requests
import geopandas as gpd
from pathlib import Path
from datetime import datetime
%matplotlib inline

In [ ]:
all_scenes_f = Path('/home/rave/serdp/rhone/rhone-ecostress/rasters/chirps-clipped')

all_precip_paths = list(all_scenes_f.glob("*"))
# for some reason the fll value is not correct. this is the correct bad value to mask by
testf = all_precip_paths[0]
x = rio.open_rasterio(testf)
badvalue = np.unique(x.where(x != x._FillValue).sel(band=1))[0]

In [ ]:
def chirps_path_date(path):
    _, _, year, month, day, _ = path.name.split(".")
    day = day.split("-")[0]
    return datetime(int(year), int(month), int(day))


def open_chirps(path):
    data_array = rio.open_rasterio(path) #chunks makes i lazyily executed
    data_array = data_array.sel(band=1).drop("band") # gets rid of old coordinate dimension since we need bands to have unique coord ids
    data_array["date"] = chirps_path_date(path) # makes a new coordinate
    return data_array.expand_dims({"date":1}) # makes this coordinate a dimension



### data is not tiled so not a good idea to use chunking
https://github.com/pydata/xarray/issues/2314

In [ ]:
import rasterio
with rasterio.open(testf) as src:
    print(src.profile)

In [ ]:
len(all_precip_paths) * 41.7 / 10e3

In [ ]:
%timeit open_chirps(testf)

In [ ]:
daily_chirps_arrs = [open_chirps(path) for path in all_precip_paths]

In [ ]:
all_chirps_arr = xr.concat(daily_chirps_arrs, dim="date")

In [ ]:
testf1 = all_precip_paths[1]

y = rio.open_rasterio(testf1, chunks={'band': 1})
y.where(y != badvalue).sel(band=1).plot.imshow()

In [ ]:
import pandas as pd

all_scene_ids = [str(i) for i in list(all_scenes_f.glob("L*"))]
df = pd.DataFrame({"scene_id":all_scene_ids}).reindex()
split_vals_series = df.scene_id.str.split("/")

dff = pd.DataFrame(split_vals_series.to_list(), columns=['_', '__', '___', '____', '_____', '______', 'fname'])

df['date'] = dff['fname'].str.slice(10,18)

df['pathrow'] = dff['fname'].str.slice(4,10)

df['sensor'] = dff['fname'].str.slice(0,4)

df['datetime'] = pd.to_datetime(df['date'])

df = df.set_index("datetime").sort_index()

In [ ]:
marc_df = df['2014-01-01':'2019-12-31']

In [ ]:
marc_df = marc_df[marc_df['sensor']=="LC08"]

In [ ]:

x.where(x != badvalue).sel(band=1).plot.imshow()